In [2]:
import pandas as pd

please change the file path for your computer
I have kept required files in folder as well.

In [3]:
moviepath='/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/movies.dat'

In [4]:
userpath='/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/users.dat'

In [5]:
ratingpath='/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/ratings.dat'

In [6]:
moviefile= pd.read_table(moviepath,sep='::',names=['movie_id','movie_name', 'genre'],encoding='iso-8859-15')

<ipython-input-6-e8d27d2a03fb>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  moviefile= pd.read_table(moviepath,sep='::',names=['movie_id','movie_name', 'genre'],encoding='iso-8859-15')


In [6]:
moviefile

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
userfile= pd.read_table(userpath,sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'],encoding='iso-8859-15')
#UserID::Gender::Age::Occupation::Zip-code

<ipython-input-7-aa7db5e09aee>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  userfile= pd.read_table(userpath,sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'],encoding='iso-8859-15')


In [8]:
#userfile['UserID'].nunique()

In [8]:
ratingfile= pd.read_table(ratingpath,sep='::',names=['UserID','MovieID','Rating','Timestamp'],encoding='iso-8859-15')
#UserID::MovieID::Rating::Timestamp

<ipython-input-8-7c483d080295>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratingfile= pd.read_table(ratingpath,sep='::',names=['UserID','MovieID','Rating','Timestamp'],encoding='iso-8859-15')


In [10]:
ratingfile['MovieID'].nunique()

3706

In [11]:
moviefile['movie_id'].nunique()

3883

In [12]:
file_genre= pd.merge(left=ratingfile, right=moviefile, how='left', left_on='MovieID', right_on='movie_id') 

In [8]:
file_genre=file_genre.drop(['Timestamp','movie_id'],axis=1)

In [15]:
file_genre

,UserID,MovieID,Rating,movie_name,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
1000204,6040,1091,1,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama


In [9]:
genre_list= set()

In [10]:
for g in file_genre['genre'].str.split('|'):
    genre_list= genre_list.union(set(g))

In [11]:
for g in genre_list:
    file_genre[g]=0

In [20]:
#glist= file_genre.loc[0,'genre'].split('|')

In [12]:
i=file_genre.index.min()
#type(i)
while i<=file_genre.index.max():
    for g in file_genre.loc[i,'genre'].split('|'):
        file_genre.at[i,g]=1
    i=i+1

In [13]:
file_genre= file_genre.drop(['movie_name'], axis=1)

In [14]:
backupfile=file_genre

In [15]:
file_genre=file_genre.drop(['genre'], axis=1)

Here, i am saving the intermediate preprocessed data file so that i dont have preprocess it every time as it takes a lot of time to do one hot encoding

In [16]:
file_genre.to_csv(r'/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/file_genre',index = False)

In [9]:
file_genre= pd.read_csv('/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/file_genre')

In [10]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
from pyspark import SparkContext

In [13]:
from pyspark import SQLContext

In [14]:
sc=SparkContext()

In [15]:
sqlctx= SQLContext(sc)

In [16]:
SpData= sqlctx.createDataFrame(file_genre)

In [17]:
ignore=['UserID','MovieID', 'Rating']

In [18]:
assembler=VectorAssembler(inputCols=[x for x in file_genre if x not in ignore], outputCol='features')

In [19]:
assembler

VectorAssembler_bcb036b1dbcc

In [20]:
tdata=assembler.transform(SpData)

In [21]:
tdata

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint, Sci-Fi: bigint, Mystery: bigint, Romance: bigint, Western: bigint, Drama: bigint, Children's: bigint, Comedy: bigint, Horror: bigint, Action: bigint, Crime: bigint, Film-Noir: bigint, Fantasy: bigint, War: bigint, Animation: bigint, Musical: bigint, Thriller: bigint, Documentary: bigint, Adventure: bigint, features: vector]

In [22]:
SpData

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint, Sci-Fi: bigint, Mystery: bigint, Romance: bigint, Western: bigint, Drama: bigint, Children's: bigint, Comedy: bigint, Horror: bigint, Action: bigint, Crime: bigint, Film-Noir: bigint, Fantasy: bigint, War: bigint, Animation: bigint, Musical: bigint, Thriller: bigint, Documentary: bigint, Adventure: bigint]

In [28]:
least_ssee=[]
for i in range(25,35):
    kmeans= KMeans().setK(i).setSeed(134)
    model= kmeans.fit(tdata)
    sse=model.summary.trainingCost
    least_ssee.append([sse,i])
    

In [27]:
least_sse

[[751888.2220114515, 10],
 [763639.5500068518, 11],
 [702086.3429350647, 12],
 [695197.062979923, 13],
 [685516.4730320052, 14],
 [660556.9409607389, 15],
 [669664.1455954626, 16],
 [644173.2099429462, 17],
 [635584.0380843175, 18],
 [559569.4318789064, 19],
 [556312.5484007043, 20],
 [518024.9331328692, 21],
 [549568.9004488393, 22],
 [540707.6287568064, 23],
 [494798.03396813106, 24],
 [502123.6624947026, 25],
 [498955.09944292734, 26],
 [489590.6635881172, 27],
 [502650.16439045937, 28],
 [472141.6583442273, 29]]

In [29]:
least_ssee
#best K is 29

[[502123.6624947026, 25],
 [498955.0994429273, 26],
 [489590.6635881172, 27],
 [502650.16439045937, 28],
 [472141.6583442273, 29],
 [515729.22612895304, 30],
 [435086.11919278424, 31],
 [483245.95672973094, 32],
 [426129.84960044944, 33],
 [433239.09027276083, 34]]

In [23]:
kmeans= KMeans().setK(29).setSeed(134)

In [27]:
model=kmeans.fit(tdata)

In [28]:
prediction=model.transform(tdata)

In [28]:
type(prediction)

pyspark.sql.dataframe.DataFrame

In [29]:
prediction.show()

+------+-------+------+-----------+-------+-------+-----+-------+------+---------+---------+---------+------+---+--------+-----+----------+------+-------+-------+------+--------------------+----------+
|UserID|MovieID|Rating|Documentary|Mystery|Romance|Crime|Western|Horror|Film-Noir|Adventure|Animation|Sci-Fi|War|Thriller|Drama|Children's|Comedy|Musical|Fantasy|Action|            features|prediction|
+------+-------+------+-----------+-------+-------+-----+-------+------+---------+---------+---------+------+---+--------+-----+----------+------+-------+-------+------+--------------------+----------+
|     1|   1193|     5|          0|      0|      0|    0|      0|     0|        0|        0|        0|     0|  0|       0|    1|         0|     0|      0|      0|     0|     (18,[12],[1.0])|         5|
|     1|    661|     3|          0|      0|      0|    0|      0|     0|        0|        0|        1|     0|  0|       0|    0|         1|     0|      1|      0|     0|(18,[8,13,15],[1....|  

In [29]:
drop_col=['Documentary','Mystery','Romance','Crime','Western','Horror','Film-Noir','Adventure','Animation','Sci-Fi','War','Thriller','Drama',"Children's",'Comedy','Musical','Fantasy','Action']

In [30]:
prediction= prediction.drop(*drop_col)

In [31]:
pred_panda=prediction.select('*').toPandas()

In [32]:
dataf=prediction.groupby('UserID','prediction').agg({'Rating':'mean'})

In [33]:
dataf_panda=dataf.select('*').toPandas()

In [35]:
pred_panda.head()

,UserID,MovieID,Rating,features,prediction
0,1,1193,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5
1,1,661,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",3
2,1,914,3,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",25
3,1,3408,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5
4,1,2355,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",11


In [36]:
dataf_panda.head()

,UserID,prediction,avg(Rating)
0,1,12,4.000000
1,9,24,5.000000
2,10,27,3.666667
3,10,14,4.500000
4,24,0,4.000000


In [34]:
df= pred_panda.merge(dataf_panda,how='left', left_on=['UserID','prediction'],right_on=['UserID','prediction'])

In [35]:
df.head()

,UserID,MovieID,Rating,features,prediction,avg(Rating)
0,1,1193,5,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",5,4.357143
1,1,661,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",3,4.400000
2,1,914,3,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",25,4.000000
3,1,3408,4,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",5,4.357143
4,1,2355,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",11,4.142857


In [36]:
df=df.rename(columns={'avg(Rating)':'avgRating'})

In [39]:
rmse=((df.Rating-df.avgRating)**2).mean()**0.5

In [40]:
type(rmse)

float

In [41]:
rmse.real

0.9303716850954153